In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

In [3]:
import deepmatcher as dm
from utilities_functions.intermediate_layer_extraction import return_layer_input
from utilities_functions.ri_calculator import find_smallest_variation_to_change

In [4]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.load_state('../../models/dblp_acm_hybrid.pth')

In [5]:
testneg_classifier_inputs,testneg_ids = return_layer_input('../../Structured/DBLP-ACM/','test_negatives',
                                                           batch_size=128,model=model,layer=model.classifier,
                                                          device=-1)

In [6]:
testpos_classifier_inputs,testpos_ids = return_layer_input('../../Structured/DBLP-ACM/','test_positives',
                                                           batch_size=128,model=model,layer=model.classifier,
                                                          device=-1)

In [7]:
model = model.cuda()

In [8]:
from tqdm import tqdm

In [9]:
attributes = ['title','authors','venue','year']
## Basta scegliere un campione a caso
classifier_length = len(testneg_classifier_inputs[0][0])
attribute_length = int(classifier_length/len(attributes))

In [10]:
current_sample = 0
#each column of this matrix is related to a specific attribute
negatives_ri = []
for batch in testneg_classifier_inputs:
    for sample_index in tqdm(range(len(batch))):
        current_sample_ris = []
        for j,attribute in enumerate(attributes):
            variation = find_smallest_variation_to_change(model.classifier,
                                                             classifier_length= classifier_length,
                                                             attribute_length= attribute_length,
                                                          input_matrix=batch,
                                                             vector_index=sample_index,attributes=[j],
                                                          class_to_reach=1,lr0=1000,decay=0.005)
            current_sample_ris.append(variation)
        negatives_ri.append(current_sample_ris)
        current_sample+=1

  2%|▏         | 2/128 [00:01<01:53,  1.11it/s]


KeyboardInterrupt: 

In [ ]:
ri_norms_negative_samples = []
for ri_list in negatives_ri_matrix:
    ri_norms_negative_samples.append(list(map(lambda x:torch.norm(x).data[0],ri_list)))
negatives_variation_df = pd.DataFrame(data= ri_norms_negative_samples,columns=attributes)
ri_sums = negatives_variation_df.sum(numeric_only=True)
ri_sums.plot.bar()